## 准备数据

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [10]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        input_size = 784  
        hidden_size = 128
        output_size = 10
        self.W1 = tf.Variable(tf.random.normal([input_size, hidden_size], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([hidden_size]))
        self.W2 = tf.Variable(tf.random.normal([hidden_size, output_size], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([output_size]))
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        hidden = tf.matmul(x, self.W1) + self.b1
        hidden = tf.nn.relu(hidden)
        logits = tf.matmul(hidden, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [25]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [32]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.5062385 ; accuracy 0.10035
epoch 1 : loss 2.4768956 ; accuracy 0.102683336
epoch 2 : loss 2.4500592 ; accuracy 0.10425
epoch 3 : loss 2.4253204 ; accuracy 0.106633335
epoch 4 : loss 2.4023545 ; accuracy 0.10865
epoch 5 : loss 2.3809042 ; accuracy 0.11156667
epoch 6 : loss 2.360763 ; accuracy 0.113966666
epoch 7 : loss 2.3417637 ; accuracy 0.117
epoch 8 : loss 2.323771 ; accuracy 0.11985
epoch 9 : loss 2.306669 ; accuracy 0.12358333
epoch 10 : loss 2.2903597 ; accuracy 0.1274
epoch 11 : loss 2.2747588 ; accuracy 0.13196667
epoch 12 : loss 2.2597957 ; accuracy 0.13705
epoch 13 : loss 2.245407 ; accuracy 0.1419
epoch 14 : loss 2.2315383 ; accuracy 0.1468
epoch 15 : loss 2.2181432 ; accuracy 0.15266667
epoch 16 : loss 2.2051828 ; accuracy 0.1584
epoch 17 : loss 2.1926188 ; accuracy 0.16478333
epoch 18 : loss 2.180419 ; accuracy 0.17075
epoch 19 : loss 2.1685565 ; accuracy 0.17696667
epoch 20 : loss 2.1570034 ; accuracy 0.18351667
epoch 21 : loss 2.1457415 ; accuracy 0.1912